# Jupyter Notebook para implenetação teste de Transfer Learning
### Algoritmo utilizando apenas 2% do conjutno de dados

#### Importação de bibliotecas

In [1]:
from keras import applications
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Flatten
import numpy as np
from sklearn.model_selection import train_test_split
from keras import backend as K

C:\Users\giovanadelucca\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Leitura do conjunto de dados

In [2]:
L = np.load('L.npy')
ab = np.load('ab.npy')

#### Separação do conjunto de dados para treinamento e testes

In [3]:
x_train, x_test, y_train, y_test = train_test_split(L,ab,test_size=0.3)

#### Modelo de Transfer Learning 

In [4]:
model = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(128,128,3), pooling=None)

#### Ajustes do modelo

In [7]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
predictions = Dense(32768, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile]the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=[f1,"accuracy"])

#### Treinamento do modelo

In [8]:
model_final.fit(x_train,y_train, epochs=3, verbose=2)

Epoch 1/3


KeyboardInterrupt: 

#### Implementação da função f-score

In [6]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#### Métricas de desempenho

In [11]:
loss, f_score, accuracy = model.evaluate(x_test, y_test)

net = 'REDE'
out_loss = '\nLoss: '+str(loss)
out_f_score = '\nF-score: '+str(f_score)
out_accuracy = '\nAccuracy: '+str(accuracy)

out = net+out_loss+out_f_score+out_accuracy

#### Salvando resultados em um arquivo

In [12]:
arq = open('output.txt', 'w')
arq.write(out)
arq.close()